In [ ]:
---
layout: post
categories: [Overworld Lessons]
title: Overworld Level Profiles 
description:  Programmer Profiles 
type: issues 
comments: true
permalink: /overworld/EnemyCollision1
---

# Enemy Collisions

Our game is already filled with different types of collision reactions with NPCs. However, in order to make our game even more challenging, we can introduce the idea of having an Enemy. Having an enemy doesn't come without its consequences. In this lesson, you will learn how to create consequences when our player collides with an enemy. 